# Generate stores & customers for the demo

## Setup the environment

### Install packages

In [3]:
!pip install faker
!pip install pymorton
!pip install v3io
!pip install v3io_frames

    100% |################################| 194kB 20.7MB/s 
  Running setup.py bdist_wheel for v3io ... done
  Stored in directory: /igz/.cache/pip/wheels/16/06/30/71478da3b7ae64df011a3466bcea2653a35ba3c5aa43ca1331
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /igz/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built v3io ujson


### Define environment variables

In [2]:
# Stores
NUMBER_OF_STORES=500
STORES_TABLE='stores'
ACCURACY=20
BASE_ACCURACY=14

# Customers
NUMBER_OF_CUSTOMERS=3000
CUSTOMERS_TABLE='customers'
CUSTOMERS_DATA_FILE='data/trained/data_train_n.csv'

### imports

In [3]:
import logging
import os
import pickle
import random
import itertools

# Demo
import pandas as pd
import pymorton as pm
from faker import Faker
from faker.providers import BaseProvider

# DB
import v3io as v3c
import v3io_frames as v3f

### Open DB client

In [4]:
client = v3f.Client('http://'+os.getenv('V3IO_FRAMESD'))

## Generate Stores

### Location provider

In [4]:
class LocationProvider(BaseProvider):
    '''
    Creates locations within base_location

    Uses QuadTree for Geohashing
        @{http://tech.taskrabbit.com/blog/2015/06/09/elasticsearch-geohash-vs-geotree/}
        @{http://mapzen.github.io/leaflet-spatial-prefix-tree/}
        @{http://blog.notdot.net/2009/11/Damn-Cool-Algorithms-Spatial-indexing-with-Quadtrees-and-Hilbert-Curves}
    '''    
    def location(self, location_base: str, base_acc=10, acc=20):
        coordinates = location_base[:base_acc]
        for i in range(acc-len(coordinates)):
            coordinates += str(random.randint(0, 3))
        return coordinates

Add the location provider as a faker provider

In [5]:
faker = Faker()
faker.add_provider(LocationProvider)

### Define scenario environment (locations)

In [6]:
london_city = (51.514926, -0.089580)
london_city_south = (51.501593, -0.094942)
london_west = (51.512309, -0.128966)
london_south_west = (51.495022, -0.162268)

london_coordinates = []
london_coordinates.append(london_city)
london_coordinates.append(london_city_south)
london_coordinates.append(london_west)
london_coordinates.append(london_south_west)

london_coordinates_qt = list(map(lambda cooridnate: pm.interleave_latlng(*cooridnate), london_coordinates))
london_coordinates_qt

['03311311313011311011000321002320',
 '03311311311233323013031101320003',
 '03311311313010023000032330133111',
 '03311311311222300331010333220231']

### Define Store class

In [11]:
class Store(object):
    def __init__(self, store_id: int, name: str, location: str):
        self.store_id = store_id if store_id is not None else random.randint(1, int(os.genev('NUMBER_OF_STORES', 500)))
        self.location = location
        (self.lat, self.long) = pm.deinterleave_latlng(location)
        self.store_name = name
        self.customers = 0
    
    def json(self):
        json_store = {
            'id': self.store_id,
            'name': self.store_name,
            'location': self.location,
            'latitude': self.lat,
            'longitude': self.long,
            'customers': self.customers
        }
        return json_store

### Create stores

In [12]:
def create_stores(faker, number_of_stores: int=500):
    remaining = number_of_stores
    stores = []
    while remaining > 0:
        # Create stores
        stores_batch = [Store(faker.msisdn(), faker.company(), faker.location(random.choice(london_coordinates_qt), BASE_ACCURACY, ACCURACY)).json() 
              for i in range(remaining)]
        
        # Count for duplicate locations
        locations = list(map(lambda store: store['location'], stores_batch))
        locations = list(dict.fromkeys(locations))
        remaining -= len(locations)
        
        # Update stores list
        stores.append(stores_batch)

    stores = list(itertools.chain.from_iterable(stores))
    stores = pd.DataFrame.from_records(stores)
    stores = stores.set_index(['location'])
    stores = stores[~stores.index.duplicated(keep='first')]
    return stores

In [34]:
client.delete('kv', STORES_TABLE)

In [35]:
stores = create_stores(faker, NUMBER_OF_STORES)
stores.head(5)

,customers,id,latitude,longitude,name
location,,,,,
03311311311222303110,0,3506253484627,51.495667,-0.159988,Caldwell LLC
03311311311222012301,0,4541826602235,51.486053,-0.168571,Stewart Inc
03311311313011210102,0,8350158735936,51.515236,-0.102997,Williams Ltd
03311311311222213010,0,4109725832326,51.495667,-0.166855,Melendez-Nichols
03311311313010100031,0,0282938785932,51.504593,-0.119820,Hernandez-Cole


### Upload to DB 

In [36]:
# client.delete('kv', STORES_TABLE)
client.write('kv', STORES_TABLE, stores)

{'num_frames': 1, 'num_rows': 500}

In [37]:
client.read('kv', STORES_TABLE).head(5)

,customers,id,latitude,longitude,name
__name,,,,,
03311311313011130223,0.0,8516057858241,51.511803,-0.093040,Robinson-Brown
03311311313011332203,0.0,6944538649976,51.524849,-0.093040,Walton Inc
03311311311222330311,0.0,6661569629685,51.499786,-0.156898,Smith PLC
03311311311222221033,0.0,6242907425574,51.499443,-0.172005,Williams PLC
03311311311222033021,0.0,5777950516357,51.490860,-0.167198,Hanson and Sons


## Generate customers

In [1]:
class Customer(object):
    def __init__(self, id=0, location='0', products=""):
        self.id = id
        self.location = location
        self.products = products
        
    def json(self):
        json_customer = {
            'id': self.id,
            'location': self.location,
            'products': self.products
        }
        return json_customer

In [5]:
def create_customers(customers_data: pd.DataFrame, number_of_customers: int):
    available_customer = list(customers_data.user_id)
    customers = [Customer(id=i,
                          products=customers_data.loc[customers_data['user_id'] == random.choice(available_customer)]\
                                    .drop(['user_id', 'Unnamed: 0'], axis=1)\
                                    .to_json()).json() for i in range(number_of_customers)]
    customers = pd.DataFrame.from_records(customers)
    customers.set_index(['id'])
    return customers

In [6]:
# Load customers seed data
customers_data = pd.read_csv(CUSTOMERS_DATA_FILE)

# Create actual customers
customers = create_customers(customers_data, NUMBER_OF_CUSTOMERS)
customers.head(5)

,id,location,products
0,0,0,"{""user_product_reordered_ratio"":{""45810"":1.0,""..."
1,1,0,"{""user_product_reordered_ratio"":{""4631"":1.0,""7..."
2,2,0,"{""user_product_reordered_ratio"":{""6572"":1.0,""2..."
3,3,0,"{""user_product_reordered_ratio"":{""119769"":1.0,..."
4,4,0,"{""user_product_reordered_ratio"":{""123106"":1.0,..."


## Upload to DB

In [7]:
client.delete('kv', CUSTOMERS_TABLE)
client.write('kv', CUSTOMERS_TABLE, customers)

Error: cannot call API - write error: can't wait for completion: Failed PUT with status 400

In [8]:
client.read('kv', CUSTOMERS_TABLE).shape

(2983, 3)

## Generate predictions

In [38]:
client.delete('tsdb', 'predictions')
client.create('tsdb', 'predictions', attrs={'rate': '1/s'})

In [ ]:
client.delete('')